## Five Rules of Effective Prompt Engineering
1. Clear instructions improve relevance.
2. Examples stabilize the response.
3. A defined answer format = a predictable outcome.
4. Breaking work into steps = better and more complete solutions.
5. Testing and verification = safety and correctness.

In [ ]:
# Import libraries
from dotenv import load_dotenv
load_dotenv()

from langchain_openai import ChatOpenAI

# Default model
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

### Rule 1 - Clear instructions

In [19]:
# Bad prompt — vague, no role or expectations
bad_prompt = "Write a function in Python."
print("=== Bad prompt ===")
print(llm.invoke(bad_prompt).content)

# Good prompt — clearly defined role and expectations
good_prompt = """You are an expert Python programmer.
Write a Python function that takes a list of integers
and returns a new list containing only even numbers.
Add a unit test in pytest."""
print("\n=== Correct prompt ===")
print(llm.invoke(good_prompt).content)

=== Zły prompt ===
Oczywiście! Jaką funkcję chciałbyś, abym napisał? Czy masz na myśli konkretną funkcjonalność lub problem, który chcesz rozwiązać?

=== Dobry prompt ===
Oczywiście! Oto przykładowa funkcja w Pythonie, która przyjmuje listę liczb całkowitych i zwraca nową listę zawierającą tylko liczby parzyste. Dodatkowo, dołączam test jednostkowy z użyciem biblioteki `pytest`.

### Funkcja do filtrowania liczb parzystych

```python
def filter_even_numbers(numbers):
    """Funkcja zwraca listę liczb parzystych z podanej listy."""
    return [num for num in numbers if num % 2 == 0]
```

### Test jednostkowy z użyciem pytest

Aby przetestować powyższą funkcję, możemy stworzyć plik testowy, na przykład `test_filter_even_numbers.py`:

```python
import pytest
from your_module import filter_even_numbers  # Zmodyfikuj 'your_module' na rzeczywistą nazwę pliku z funkcją

def test_filter_even_numbers():
    assert filter_even_numbers([1, 2, 3, 4, 5, 6]) == [2, 4, 6]
    assert filter_even_numbe

### Zasada 2 - Use examples

In [20]:
# Zadanie: wygeneruj tagi na podstawie treści strony firmy

# Zero-shot
zero_shot = """Podaj tagi opisujące firmę na podstawie tekstu strony:
Firma Lego produkuje zabawki dla dzieci."""
print("=== Zero-shot ===")
print(llm.invoke(zero_shot).content)

# One-shot
one_shot = """Podaj maksymalnie trzy tagi opisujące firmę na podstawie tekstu strony.
Przykład:
Tekst: Firma Lego produkuje klocki dla dzieci.
Tagi: zabawki, klocki, dzieci

Teraz:
Tekst: Firma Nike produkuje odzież i buty sportowe.
Tagi:"""
print("\n=== One-shot ===")
print(llm.invoke(one_shot).content)

# Few-shot
few_shot = """Podaj maksymalnie trzy tagi opisujące firmę na podstawie tekstu strony.

Przykład 1:
Tekst: Firma Lego produkuje klocki dla dzieci.
Tagi: zabawki, klocki, dzieci

Przykład 2:
Tekst: Firma Nike produkuje odzież i buty sportowe.
Tagi: sport, odzież, obuwie

Teraz:
Tekst: Firma Tesla produkuje samochody elektryczne i magazyny energii.
Tagi:"""
print("\n=== Few-shot ===")
print(llm.invoke(few_shot).content)


=== Zero-shot ===
Oto tagi, które mogą opisać firmę Lego na podstawie podanego tekstu:

- Lego
- zabawki
- dzieci
- produkcja
- kreatywność
- edukacja
- rozwój
- rozrywka
- konstrukcyjne
- innowacja

=== One-shot ===
odzież, buty, sport

=== Few-shot ===
samochody, elektryczność, energia


In [21]:
prompt = """Stwórz listę 5 sloganów reklamowych dla kawiarni.
Zastosuj styl z przykładowych sloganów załączonych poniżej:
"U nas dostaniesz kawę na ławę!"
"Kawa szybko tanio i konkretnie!"
"""
print(llm.invoke(prompt).content)

Oto pięć sloganów reklamowych dla kawiarni w podobnym stylu:

1. "Kawa z sercem, relaks z duszą!"
2. "Szybka kawa, wolny czas!"
3. "Kawowe przysmaki na każdą chwilę!"
4. "Wpadnij po kawę, wyjdź z uśmiechem!"
5. "Kawa w kubku, radość na twarzy!"


In [22]:
prompt = """Stwórz listę 5 sloganów reklamowych dla kawiarni.
Zastosuj styl z przykładowych sloganów załączonych poniżej:
"Nasza kawa jest niczym spacer w rajskim ogrodzie w słoneczny radosny dzień."
"Pijąc naszą kawę poczujesz się tak, jakby anioł zatańczył Tobie na języku."
"""
print(llm.invoke(prompt).content)

1. "Nasza kawa to magiczna podróż w głąb lasu, gdzie każdy łyk to spotkanie z naturą."
2. "Zanurz się w aromacie naszej kawy, jakbyś odkrywał tajemnice skarbu ukrytego w sercu gór."
3. "Pij naszą kawę i poczuj, jak każdy łyk to melodia, która rozbrzmiewa w rytmie Twojego serca."
4. "Nasza kawa to słodka chwila w ramionach porannego słońca, która rozbudzi Twoje najpiękniejsze marzenia."
5. "Z każdym łykiem naszej kawy przeniesiesz się na plażę, gdzie fale szepczą historie z dalekich krain."


### Zasada 3 - Zdefiniuj format odpowiedzi
walidacja wejścia i wyjścia z wykorzystaniem biblioteki Pydantic

In [23]:
from pydantic import BaseModel, Field
from typing import List
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
import dotenv

dotenv.load_dotenv()

# Definicja schematu wyjścia (Pydantic)
class CityGuide(BaseModel):
    city: str = Field(..., description="Miasto, którego dotyczy przewodnik")
    summary: str = Field(..., description="Krótki opis miasta (2–3 zdania)")
    must_do: List[str] = Field(..., description="Lista 3–5 rzeczy do zrobienia")

# LLM + structured output
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
structured_llm = llm.with_structured_output(CityGuide)

prompt = ChatPromptTemplate.from_messages([
    ("system", "Jesteś ekspertem od podróży. Odpowiadaj po polsku, zwięźle."),
    ("user", "Stwórz krótki przewodnik po {city} dla {days}-dniowej wizyty.")
])

chain = prompt | structured_llm

# Walidacja wejścia Pydantic + wywołanie łańcucha
from pydantic import BaseModel, Field

class GuideRequest(BaseModel):
    city: str = Field(min_length=2)
    days: int = Field(ge=1, le=7)

req = GuideRequest(city="Wrocław", days=2)

result: CityGuide = chain.invoke(req.model_dump())  # <- dostajesz OBIEKT Pydantic
print(result)                         # CityGuide(city=..., summary=..., must_do=[...])
print(result.model_dump_json(indent=2))  # JSON gotowy do zapisu/transportu

city='Wrocław' summary='Wrocław to malownicze miasto w Polsce, znane z pięknej architektury, licznych mostów i bogatej historii. Jest to miejsce, gdzie tradycja spotyka nowoczesność, a tętniące życiem rynki i urokliwe uliczki przyciągają turystów z całego świata.' must_do=['Odwiedź Rynek i zobacz Ratusz oraz kolorowe kamienice.', 'Przejdź się po Ostrowie Tumskim i zwiedź Katedrę św. Jana Chrzciciela.', 'Zobacz Panoramę Racławicką, monumentalne dzieło sztuki.', 'Spędź czas w Ogrodzie Japońskim i Parku Szczytnickim.', 'Zrób zdjęcie z wrocławskimi krasnalami, które są rozsiane po całym mieście.']
{
  "city": "Wrocław",
  "summary": "Wrocław to malownicze miasto w Polsce, znane z pięknej architektury, licznych mostów i bogatej historii. Jest to miejsce, gdzie tradycja spotyka nowoczesność, a tętniące życiem rynki i urokliwe uliczki przyciągają turystów z całego świata.",
  "must_do": [
    "Odwiedź Rynek i zobacz Ratusz oraz kolorowe kamienice.",
    "Przejdź się po Ostrowie Tumskim i zwie

### Zasada 4 - Dziel złożone zadania na kroki

#### Przygotowanie planu wycieczki

In [24]:
# Bad prompt — everything at once
bad_prompt = """Prepare a three‑day sightseeing plan for Poznań with a budget of 300 euros,
including attractions, restaurants, transport, and maps."""
print("=== Zły prompt ===")
print(llm.invoke(bad_prompt).content[:600], "...")

# Good prompt — step by step
good_step1 = "Wypisz najważniejsze atrakcje kulturalne w Poznaniu z godzinami otwarcia."
step1 = llm.invoke(good_step1).content
print("\n=== Dobry prompt — krok 1 ===")
print(step1[:600], "...")

good_step2 = f"Na podstawie tej listy ułóż plan zwiedzania na 3 dni, maks 4 atrakcje dziennie. Atrakcje: {step1}"
step2 = llm.invoke(good_step2).content
print("\n=== Dobry prompt — krok 2 ===")
print(step2[:600], "...")


=== Zły prompt ===
Oto trzydniowy plan zwiedzania Poznania z budżetem 300 euro. Plan uwzględnia atrakcje turystyczne, restauracje, transport oraz mapy.

### Dzień 1: Stare Miasto i okolice

**Rano:**
- **Śniadanie:** Kawiarnia "Café La Ruina" (około 5 euro)
- **Atrakcja:** Stary Rynek, Ratusz (bezpłatnie)
- **Transport:** Spacer po Starym Mieście (bezpłatnie)

**Południe:**
- **Atrakcja:** Muzeum Narodowe (bilet wstępu około 5 euro)
- **Obiad:** Restauracja "Bamberka" (około 10 euro)

**Popołudnie:**
- **Atrakcja:** Ostrów Tumski, Katedra (bezpłatnie)
- **Transport:** Spacer po Ostrowie Tumskim (bezpłatnie)

**W ...

=== Dobry prompt — krok 1 ===
Oto lista najważniejszych atrakcji kulturalnych w Poznaniu wraz z ich godzinami otwarcia (godziny mogą się różnić w zależności od sezonu, dlatego zawsze warto sprawdzić aktualne informacje przed wizytą):

1. **Stary Rynek**  
   - Opis: Serce Poznania, z pięknymi kamienicami i ratuszem.
   - Godziny otwarcia: Całodobowo (rynek jest dostępny pr

#### Step‑by‑step article generation

In [25]:
ARTICLE_SUBJECT = "Jakie kompetencje będą najcenniejsze u programistów za 10 lat."

plan = f"Przygotuj plan składający sie z 3 punktów artykułu na temat: {ARTICLE_SUBJECT}."
planGenerated = llm.invoke(plan).content
print(planGenerated, "...")

Oto plan artykułu na temat "Jakie kompetencje będą najcenniejsze u programistów za 10 lat":

### 1. Rozwój umiejętności w zakresie sztucznej inteligencji i uczenia maszynowego
   - **Opis**: Wprowadzenie do rosnącej roli AI i ML w różnych branżach. 
   - **Przykłady**: Zastosowania w automatyzacji, analityce danych i tworzeniu inteligentnych systemów.
   - **Wnioski**: Dlaczego programiści muszą znać te technologie, aby pozostać konkurencyjni na rynku pracy.

### 2. Zrozumienie architektury chmurowej i DevOps
   - **Opis**: Znaczenie chmury obliczeniowej i praktyk DevOps w nowoczesnym rozwoju oprogramowania.
   - **Przykłady**: Narzędzia i platformy chmurowe (AWS, Azure, Google Cloud) oraz metodyki CI/CD.
   - **Wnioski**: Jak umiejętności związane z chmurą i DevOps mogą zwiększyć efektywność i elastyczność zespołów programistycznych.

### 3. Umiejętności miękkie i współpraca w zespole
   - **Opis**: Rola umiejętności interpersonalnych w pracy programisty, w tym komunikacja, empatia i 

In [26]:
MAX_WORDS_IN_PARAGRAPH = 200

prevParagraphContent = ""

for i in range(1,4):
    paragraphPrompt = f"Przygotuj tylko treść {i}. akapitu artykułu: {ARTICLE_SUBJECT}. Maksymalna długość tekstu to {MAX_WORDS_IN_PARAGRAPH}. Wygeneruj tylko tekst jednego paragrafu. Plan całego artykułu:\n {planGenerated}. Treść poprzednich akapitów: {prevParagraphContent}."
    paragraphContent = llm.invoke(paragraphPrompt).content
    prevParagraphContent += paragraphContent + "\n"
    print(f"Akapit {i}.", paragraphContent, "\n\n")

Akapit 1. W ciągu najbliższych dziesięciu lat umiejętności związane z sztuczną inteligencją (AI) i uczeniem maszynowym (ML) staną się kluczowe dla programistów, którzy pragną pozostać konkurencyjni na rynku pracy. W miarę jak technologie te zyskują na znaczeniu w różnych branżach, ich zastosowania w automatyzacji procesów, analityce danych oraz tworzeniu inteligentnych systemów będą się intensyfikować. Programiści, którzy opanują te umiejętności, będą w stanie nie tylko tworzyć bardziej zaawansowane aplikacje, ale także przyczyniać się do innowacji w swoich organizacjach. W obliczu rosnącej automatyzacji i potrzeby przetwarzania ogromnych zbiorów danych, znajomość AI i ML stanie się nie tylko atutem, ale wręcz koniecznością w codziennej pracy programisty. 


Akapit 2. W kontekście przyszłości programowania, zrozumienie architektury chmurowej oraz praktyk DevOps będzie miało kluczowe znaczenie. Chmura obliczeniowa, z jej elastycznością i skalowalnością, staje się fundamentem nowoczesnyc

### Rule 5 - Test and verify results

In [1]:
from langchain_classic.evaluation import load_evaluator
from dotenv import load_dotenv

load_dotenv()

evaluator = load_evaluator("embedding_distance", embeddings_model="openai")

result = evaluator.evaluate_strings(
    prediction="Stolica Polski to Warszawa",
    reference="coś zupełnie innego"
)

print(result)


{'score': 0.2288585785726367}
